# Google Colab Rust Setup

The following cell is used to set up and spin up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 

In [ ]:
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## Iterators and Closures

### Anatomy of an Iterator

In [3]:
fn is_prime(n: u32) -> bool {
    if n <= 1 {
        return false;
    }
    for i in 2..(n / 2 + 1) {
        if n % i == 0 {
            return false;
        }
    }
    true
}

fn sum_of_primes(n: u32) -> u32 {
    (2..).filter(|&x| is_prime(x)).take(n as usize).sum()
}

let n = 10;
let result = sum_of_primes(n);
println!("The sum of the first {} prime numbers is: {}", n, result);

The sum of the first 10 prime numbers is: 129


### Key Traits: Iterator and IntoIterator

In [4]:
struct MyCollection {
    data: Vec<i32>,
}

impl MyCollection {
    fn new() -> Self {
        MyCollection { data: Vec::new() }
    }

    fn add(&mut self, value: i32) {
        self.data.push(value);
    }
}

impl IntoIterator for MyCollection {
    type Item = i32;
    type IntoIter = std::vec::IntoIter<Self::Item>;

    fn into_iter(self) -> Self::IntoIter {
        self.data.into_iter()
    }
}


let mut collection = MyCollection::new();

collection.add(42);
collection.add(100);
collection.add(7);

for item in collection {
    println!("Item: {}", item);
}

Item: 42
Item: 100
Item: 7


()

### Creating Iterators with `from_fn` and `successors`

In [7]:
:dep rand = {version="0.8.5"}

In [9]:
use rand::random;

use std::iter::from_fn;

let lengths: Vec<f64> =
    from_fn(|| Some((random::<f64>() - random::<f64>()).abs()))
    .take(1000)
    .collect();
lengths[..10]

[0.026231996097305532, 0.3638420476998353, 0.529187247261019, 0.3697265864723026, 0.9199704902784379, 0.0578703524231724, 0.2434883502345343, 0.1948597360516151, 0.38109952692955174, 0.47026570340584917]

In [11]:
:dep num = {version="0.4.1"}

In [13]:
use num::Complex;
use std::iter::successors;

fn escape_time(c: Complex<f64>, limit: usize) -> Option<usize> {
    let zero = Complex { re: 0.0, im: 0.0 };

    let result = successors(Some(zero), |&z| Some(z * z + c))
        .take(limit)
        .enumerate()
        .find(|(_i, z)| z.norm_sqr() > 4.0);

    match result {
        Some((i, _z)) => Some(i),
        None => None,
    }
}

let c = Complex { re: -0.7, im: 0.27015 };
let limit = 1000;

match escape_time(c, limit) {
    Some(steps) => println!("Escaped after {} steps.", steps),
    None => println!("Did not escape within {} steps.", limit),
}

Escaped after 96 steps.


()

In [14]:
fn fibonacci() -> impl Iterator<Item = usize> {
    let mut state = (0, 1);
    std::iter::from_fn(move || {
        state = (state.1, state.0 + state.1);
        Some(state.0)
    })
}

let result: Vec<usize> = fibonacci().take(8).collect();

let expected = vec![1, 1, 2, 3, 5, 8, 13, 21];

assert_eq!(result, expected);

println!("Fibonacci sequence (first 8 numbers): {:?}", result);

Fibonacci sequence (first 8 numbers): [1, 1, 2, 3, 5, 8, 13, 21]


### Drain Methods

In [17]:
let mut outer = "Earth".to_string();

let inner = String::from_iter(outer.drain(1..4));

assert_eq!(outer, "Eh");

inner

"art"

### Iterator Adapter Methods

In [18]:
let numbers = vec![1, 2, 3, 4, 5];

let squared: Vec<i32> = numbers.into_iter().map(|x| x * x).collect();

squared

[1, 4, 9, 16, 25]

In [19]:
let numbers = 1..=10;

let evens: Vec<i32> = numbers.into_iter().filter(|x| x % 2 == 0).collect();

evens

[2, 4, 6, 8, 10]

### Building Custom Iterators

In [21]:
struct PrimeGenerator {
    current: u32,
}

impl Iterator for PrimeGenerator {
    type Item = u32;

    fn next(&mut self) -> Option<Self::Item> {
        loop {
            self.current += 1;
            if is_prime(self.current) {
                return Some(self.current);
            }
        }
    }
}

fn is_prime(num: u32) -> bool {
    if num <= 1 {
        return false;
    }

    for i in 2..(num / 2 + 1) {
        if num % i == 0 {
            return false;
        }
    }

    true
}

let prime_sequence = PrimeGenerator { current: 1 };

for prime in prime_sequence.take(10) {
    println!("{}", prime);
}

2
3
5
7
11
13
17
19
23
29


()

### Lazy Evaluation

In [22]:
let numbers = vec![1, 2, 3, 4, 5];

let doubled: Vec<i32> = numbers.iter().map(|x| x * 2).collect();

doubled

[2, 4, 6, 8, 10]

### Advanced Techniques

#### flat_map

In [23]:
let words = vec!["Rust", "is", "awesome"];

let letters: Vec<char> = words.iter()
    .flat_map(|&word| word.chars())
    .collect();

letters

['R', 'u', 's', 't', 'i', 's', 'a', 'w', 'e', 's', 'o', 'm', 'e']

#### take and skip

In [24]:
let numbers = vec![1, 2, 3, 4, 5, 6, 7, 8, 9, 10];

let selected_numbers: Vec<i32> = numbers.iter()
    .skip(3)
    .take(4)
    .cloned()
    .collect();

selected_numbers

[4, 5, 6, 7]

#### peekable

In [25]:
let mut numbers = vec![1, 2, 3, 4, 5].into_iter().peekable();

if let Some(&next_number) = numbers.peek() {
    println!("The next number is: {}", next_number);
} else {
    println!("No more numbers to process.");
}

let _ = numbers.next();

if let Some(&next_number) = numbers.peek() {
    println!("The next number is: {}", next_number);
} else {
   println!("No more numbers to process.");
}

The next number is: 1
The next number is: 2


()

#### filter_map

In [29]:
use std::str::FromStr;

let text = "The numbers are: 42, 19, invalid, 27, 33";

let parsed_numbers: Vec<i32> = text
    .split(',')
    .map(str::trim)
    .filter_map(|s| {
        let s = s
            .chars()
            .skip_while(|&c| !c.is_digit(10))
            .collect::<String>();
        if !s.is_empty() {
            i32::from_str(&s).ok()
        } else {
            None
        }
    })
    .collect();

parsed_numbers

[42, 19, 27, 33]

#### fuse

In [30]:
struct Unpredictable(bool);

impl Iterator for Unpredictable {
    type Item = &'static str;

    fn next(&mut self) -> Option<Self::Item> {
        if self.0 {
            self.0 = false;
            Some("totally the last item")
        } else {
            self.0 = true;
            None
        }
    }
}

let mut unpredictable = Unpredictable(true);
assert_eq!(unpredictable.next(), Some("totally the last item"));
assert_eq!(unpredictable.next(), None);
assert_eq!(unpredictable.next(), Some("totally the last item"));

let mut reliable = Unpredictable(true).fuse();
assert_eq!(reliable.next(), Some("totally the last item"));
assert_eq!(reliable.next(), None);
assert_eq!(reliable.next(), None);

#### flatten

In [31]:
use std::collections::HashMap;

let mut student_courses = HashMap::new();
student_courses.insert("Alice", vec!["Math", "Physics"]);
student_courses.insert("Bob", vec!["Computer Science", "Chemistry"]);
student_courses.insert("Charlie", vec!["Physics", "Biology"]);

let course_enrollment: HashMap<&str, usize> = student_courses
    .values()
    .flatten()
    .fold(HashMap::new(), |mut enrollment, course| {
        *enrollment.entry(course).or_insert(0) += 1;
        enrollment
    });

for (course, count) in &course_enrollment {
    println!("Course: {}, Students Enrolled: {}", course, count);
}

Course: Computer Science, Students Enrolled: 1
Course: Physics, Students Enrolled: 2
Course: Biology, Students Enrolled: 1
Course: Math, Students Enrolled: 1
Course: Chemistry, Students Enrolled: 1


()

---

### Closures

#### Variable Capture in Closures

#### Borrowing Variables

In [32]:
let multiplier = 5;
let numbers = vec![1, 2, 3, 4, 5];

let multiplied: Vec<i32> = numbers.iter().map(|x| x * multiplier).collect();

multiplied

[5, 10, 15, 20, 25]

#### Moving Variables

In [35]:
fn main() {
    let message = "Hello, Rust!".to_string();

    let greet = || {
        let message = message;
        println!("{}", message);
    };

    greet();

    // greet();
}

main()

Hello, Rust!


()

### Closures and Fn, FnMut, FnOnce Traits

#### Fn Closures

In [38]:
fn main() {
    let counter = 0;

    let print_counter = || {
        println!("Counter: {}", counter);
    };

    print_counter();

    // counter += 1;
}

main()

Counter: 0


()

#### FnMut Closures

In [39]:
fn main() {
    let mut counter = 0;
    let mut increment = || {
        counter += 1; // <3>
        println!("Counter: {}", counter);
    };

    increment();
    increment();
}

main()

Counter: 1
Counter: 2


()

#### FnOnce Closures

In [40]:
fn main() {
    let numbers = vec![1, 2, 3, 4, 5];
    let consume_numbers = || {
        let numbers = numbers;
        let sum: i32 = numbers.iter().sum();
        println!("Sum: {}", sum);
    };

    consume_numbers();
    // consume_numbers();  // <7>
}

main()

Sum: 15


()

### Working with Iterators for Efficient Data Processing

####  Accumulating Data with `fold`

In [41]:
let numbers = vec![1, 2, 3, 4, 5];

let sum: i32 = numbers.iter().fold(0, |acc, x| acc + x);

sum

15

#### Chaining Iterators with `chain`

In [42]:
fn main() {
    let numbers1 = vec![1, 2, 3];
    let numbers2 = vec![4, 5, 6];

    let combined: Vec<i32> = numbers1
        .iter()
        .chain(numbers2.iter())
        .map(|x| x * 2)
        .collect();

    println!("Combined and doubled: {:?}", combined);
}

main()

Combined and doubled: [2, 4, 6, 8, 10, 12]


()

---

### Applying Iterators and Closures to Practical Examples

#### Use Case 1: Text Analysis

In [43]:
let text = "Rust is a language that makes it easy to learn Rust and Rust programming is fun when you learn Rust";

let word_freq: HashMap<String, u32> = text
    .split_whitespace()
    .map(|word| word.to_lowercase())
    .filter(|word| !word.is_empty())
    .fold(HashMap::new(), |mut map, word| {
        *map.entry(word.to_string()).or_insert(0) += 1;
        map
    });

println!("Word Frequencies: {:?}", word_freq);

Word Frequencies: {"learn": 2, "fun": 1, "makes": 1, "to": 1, "you": 1, "that": 1, "and": 1, "when": 1, "it": 1, "language": 1, "easy": 1, "rust": 4, "programming": 1, "a": 1, "is": 2}


#### Use Case 2: Image Processing

In [44]:
#[derive(Debug)]
struct Pixel {
    red: u8,
    green: u8,
    blue: u8,
}

let mut image: Vec<Pixel> = vec![
    Pixel {
        red: 100,
        green: 50,
        blue: 25,
    },
    Pixel {
        red: 200,
        green: 100,
        blue: 50,
    },
    // ... more pixels
];

image.iter_mut().for_each(|pixel| {
    pixel.red = pixel.red.wrapping_mul(2);
    pixel.green = pixel.green.wrapping_mul(2);
    pixel.blue = pixel.blue.wrapping_mul(2);
});

println!("Image after processing: {:?}", image);

Image after processing: [Pixel { red: 200, green: 100, blue: 50 }, Pixel { red: 144, green: 200, blue: 100 }]


#### Use Case 3: Data Filtering and Transformation

In [46]:
#[derive(Debug)]
struct Product {
    name: String,
    price: f64,
    in_stock: bool,
}

fn main() {
    let products = vec![
        Product {
            name: String::from("Widget A"),
            price: 10.0,
            in_stock: true,
        },
        Product {
            name: String::from("Widget B"),
            price: 15.0,
            in_stock: false,
        },
        // ... more products
    ];

    let discount = 0.2;

    let discounted_products: Vec<Product> = products
        .into_iter()
        .filter(|product| product.in_stock)
        .map(|mut product| {
            product.price -= product.price * discount;
            product
        })
        .collect();

    println!("Discounted Products: {:?}", discounted_products);
}

main()

Discounted Products: [Product { name: "Widget A", price: 8.0, in_stock: true }]


()

#### Use Case 4: Processing Sensor Data

In [47]:
#[derive(Debug)]
struct SensorReading {
    timestamp: u64,
    value: f64,
}

fn main() {
    let sensor_data = vec![
        SensorReading {
            timestamp: 1,
            value: 20.0,
        },
        SensorReading {
            timestamp: 2,
            value: 22.0,
        },
        SensorReading {
            timestamp: 3,
            value: 23.0,
        },
        // ... more readings
    ];

    let threshold = 22.0;

    let anomalies: Vec<&SensorReading> = sensor_data
        .iter()
        .filter(|&reading| reading.value > threshold)
        .collect();

    println!("Anomalous Readings: {:?}", anomalies);
}

main()

Anomalous Readings: [SensorReading { timestamp: 3, value: 23.0 }]


()

#### Use Case 5: Financial Calculations

In [48]:
#[derive(Debug)]
struct Transaction {
    amount: f64,
    transaction_type: String,
}

fn main() {
    let transactions = vec![
        Transaction {
            amount: 100.0,
            transaction_type: String::from("Deposit"),
        },
        Transaction {
            amount: -50.0,
            transaction_type: String::from("Withdrawal"),
        },
        // ... more transactions
    ];

    let balance = transactions.iter().fold(0.0, |acc, transaction| {
        if transaction.transaction_type == "Deposit" {
            acc + transaction.amount
        } else {
            acc - transaction.amount
        }
    });

    println!("Final Balance: {:?}", balance);
}

main()

Final Balance: 150.0


()

#### Use Case 6: Sorting

In [49]:
struct Product {
    name: String,
    price: f64,
}

fn main() {
    let mut products = vec![
        Product {
            name: "Laptop".to_string(),
            price: 799.99,
        },
        Product {
            name: "Headphones".to_string(),
            price: 149.99,
        },
        Product {
            name: "Smartphone".to_string(),
            price: 599.99,
        },
    ];

    products.sort_by(|a, b| a.price.partial_cmp(&b.price).unwrap());

    for product in &products {
        println!("Product: {} - Price: ${}", product.name, product.price);
    }
}

main()

Product: Headphones - Price: $149.99
Product: Smartphone - Price: $599.99
Product: Laptop - Price: $799.99


()


---
---